# Pandas確認課題

このPandas確認問題は、データサイエンス100本ノックの問題で最低限必要な問題を抜粋したものですが、[Introduction_to_Pandas](./11_Introduction_to_Pandas.ipynb) に掲載されていない機能が必要な問題もあります。
初めて触るライブラリを調べながら使うというのはよくある光景です。この課題では皆さんにもそれに挑戦していただきます。  
ヒントとして検索キーワードなどを載せておくので、自力で調べながら解いてみましょう。  



## 必要モジュールのインポート

この問題で使うモジュールをインポートします．

In [5]:
import pandas as pd
import numpy as np

## データの読み込み

In [6]:
df_customer = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/customer.csv')
df_product = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/product.csv')
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')

---
## 問1. 条件抽出
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [7]:

df_receipt_select = df_receipt[(df_receipt['customer_id'] == "CS018205000001") &((df_receipt['amount'] >= 1000) | (df_receipt['quantity'] >= 5))]
df_receipt_select[['sales_ymd', 'customer_id', 'product_cd', 'quantity', 'amount']].head(5)



,sales_ymd,customer_id,product_cd,quantity,amount
36,20180911,CS018205000001,P071401012,1,2200
9843,20180414,CS018205000001,P060104007,6,600
21110,20170614,CS018205000001,P050206001,5,990
68117,20190226,CS018205000001,P071401020,1,2200
72254,20180911,CS018205000001,P071401005,1,1100


---
## 問2. ソート
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭5件を全項目表示せよ。

In [8]:
df_as = df_customer.sort_values(by='birth_day')
df_as.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
18817,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
12328,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
15682,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
15302,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
1681,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0


---
## 問3. 全件数
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [10]:
df_receipt.shape[0]


104681

## 問4. ユニーク件数
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [11]:
df_receipt['customer_id'].nunique()

8307

<details>
<summary>ヒント</summary>
「ユニーク」というのはそのまま検索に使える単語です。  
</details>

---
## 問5. 〇〇ごとに集計
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは先頭5件だけ表示せよ。

会員のみを抽出する方法は、例えば以下の2通りの方法があります。

In [9]:
df_receipt_only_member = df_receipt[~df_receipt["customer_id"].str.startswith("Z")]
df_receipt_only_member = df_receipt.query("not customer_id.str.startswith('Z')", engine="python")

In [10]:
df_receipt_only_member = df_receipt[~df_receipt["customer_id"].str.startswith("Z")]
customer_sales_sum = df_receipt_only_member.groupby('customer_id')['amount'].sum()
average_sales = customer_sales_sum.mean()
above_average_customers = customer_sales_sum[customer_sales_sum >= average_sales]

print(above_average_customers.head())

customer_id
CS001115000010    3044
CS001205000006    3337
CS001214000009    4685
CS001214000017    4132
CS001214000052    5639
Name: amount, dtype: int64


<details>
<summary>ヒント1</summary>
「pandas 要素ごと 集計」 などで今回使える機能に関する記事が見つかります。
</details>

<details>
<summary>ヒント2</summary>
メソッド名は "groupby" です。
</details>

---
## 問6. DataFrameの結合
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は先頭5件だけ表示せよ。

In [10]:
df_customer_only_member = df_customer[~df_customer["customer_id"].str.startswith("Z")]
df_customer_only_member = df_customer.query("not customer_id.str.startswith('Z')", engine="python")

In [13]:
df_customer_only_member = df_customer[~df_customer["customer_id"].str.startswith("Z")]
df_filtered_customer = df_customer_only_member[df_customer_only_member['gender_cd'] == 1]

df_receipt_grouped = df_receipt.groupby('customer_id')['amount'].sum().reset_index()

df_result = df_filtered_customer.merge(df_receipt_grouped, on='customer_id', how='left')
df_result['amount'] = df_result['amount'].fillna(0)


print(df_result.head())


      customer_id customer_name  gender_cd gender   birth_day  age postal_cd  \
0  CS021313000114        大野 あや子          1     女性  1981-04-29   37  259-1113   
1  CS031415000172       宇多田 貴美子          1     女性  1976-10-04   42  151-0053   
2  CS028811000001        堀井 かおり          1     女性  1933-03-27   86  245-0016   
3  CS001215000145         田崎 美紀          1     女性  1995-03-29   24  144-0055   
4  CS015414000103         奥野 陽子          1     女性  1977-08-09   41  136-0073   

                  address application_store_cd  application_date  \
0    神奈川県伊勢原市粟窪**********               S14021          20150905   
1     東京都渋谷区代々木**********               S13031          20150529   
2  神奈川県横浜市泉区和泉町**********               S14028          20160115   
3     東京都大田区仲六郷**********               S13001          20170605   
4      東京都江東区北砂**********               S13015          20150722   

      status_cd  amount  
0  0-00000000-0     0.0  
1  D-20100325-C  5088.0  
2  0-00000000-0     0.0  
3  6-2

<details>
<summary>ヒント1</summary>
タイトル通り 「pandas DataFrame 結合」などと調べれば必要な機能に関する記事が見つかります。  
</details>


<details>
<summary>ヒント2</summary>
"merge", "join"という似たメソッドがあります。  
今回の場合"merge"が便利でしょう。
</details>

---
## 問7. 時系列データ
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。なお、データは先頭5件を表示せよ。

In [14]:
df_customer['application_date'] = pd.to_datetime(df_customer['application_date'], format='%Y%m%d')

df_result = df_customer[['customer_id', 'application_date']].head()
print(df_result)


      customer_id application_date
0  CS021313000114       2015-09-05
1  CS037613000071       2015-04-14
2  CS031415000172       2015-05-29
3  CS028811000001       2016-01-15
4  CS001215000145       2017-06-05


<details>
<summary>ヒント1</summary>
「pandas datetime」などで該当の機能が見つかるかと思います。
</details>


<details>
<summary>ヒント2</summary>
"pd.to_datetime"というメソッドが使えるでしょう。このメソッドを適用する際ですが、for文を使わずに実装しましょう。

---
## 問8. 関数
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、結果は先頭5件を表示せよ。

In [16]:
df_receipt_only_member = df_receipt[~df_receipt["customer_id"].str.startswith("Z")]
customer_sales_sum = df_receipt_only_member.groupby('customer_id')['amount'].sum()
customer_sales_sum_log = np.log10(customer_sales_sum)
df_result = pd.DataFrame({
    'amount':customer_sales_sum,
    'customer_sales_sum_log':customer_sales_sum_log
}).head()
print(df_result)




                amount  customer_sales_sum_log
customer_id                                   
CS001113000004    1298                3.113275
CS001114000005     626                2.796574
CS001115000010    3044                3.483445
CS001205000004    1988                3.298416
CS001205000006    3337                3.523356


---
## 問9. 欠損数
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

In [18]:

missing_counts = df_product.isnull().sum()

print(missing_counts)


product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64


---
## 問10. 欠損値の除去
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [19]:
df_product_1 = df_product.dropna()

In [20]:
len(df_product), len(df_product_1)

(10030, 10023)

---
## 問11. 欠損値の穴埋め
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とせよ。補完実施後、各項目について欠損が生じていないことも確認すること。

In [22]:
unit_price_average = round(df_product['unit_price'].mean())
unit_cost_average = round(df_product['unit_cost'].mean())

df_product_2 = df_product.copy()

df_product_2['unit_price'] = df_product_2['unit_price'].fillna(unit_price_average)
df_product_2['unit_cost'] = df_product_2['unit_cost'].fillna(unit_cost_average)
missing_counts_after = df_product_2.isnull().sum()

print(missing_counts_after)


product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64


### 余談
ChatGPTやBing AIに聞けば大抵のことは教えてくれます。  
何回か入力文章を吟味しないといけないこともありますが、知らないことを調べる場合は自分で検索するよりも早いです。  
ただ、ChatGPTなどは嘘をつく場合があるので、自分でソースを参照する姿勢は必要です。  

これはBingAIの回答例です。  

![BingAIの回答例](./imgs/pandas/BingAI.png)